<h1 align=center> Global Wheat Detection with Keras RetinaNet </h1>

### This Notebook is for Training Purpose of Keras RetinaNet.
### RetinaNet is very slow as compared to F-RCNN so I've kept epochs and steps per epoch small for fast commiting purpose.
### I will make another notebook for inference Shortly.

### Credits for the EDA goes to [THIS Notebook](https://www.kaggle.com/devvindan/wheat-detection-eda)

<h3 align=center style=color:red>Upvote If you find this kernel interesting</h3>

In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from PIL import Image, ImageDraw
from ast import literal_eval
import matplotlib.pyplot as plt
import urllib
from tqdm.notebook import tqdm
import tensorflow as tf

# Installing Keras-RetinaNet 

In [2]:
!git clone https://github.com/fizyr/keras-retinanet.git

fatal: destination path 'keras-retinanet' already exists and is not an empty directory.


In [3]:
%cd keras-retinanet/
!pip3 install .
!python3 setup.py build_ext --inplace
%cd ../

/home/tiago/PycharmProjects/GlobalWheatDetectionKaggle/keras-retinanet
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Processing /home/tiago/PycharmProjects/GlobalWheatDetectionKaggle/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=160130 sha256=ad6d024710d9a7590ba591f876d8feeff1a31f0fb7e92bb0a548b23d22c8f0d9
  Stored in directory: /home/tiago/.cache/pip/wheels/29/a5/6c/5d6e071081544585100fc7b3ce7dfa3f843e8b7d0cce433301
Successfully built keras-retinanet
  Attempting uninstall: keras-retinanet
    Found existing installation: keras-retinanet 0.5.1
    Uninstalling keras-retinanet-0.5.1:
      Successfully uninstalled keras-retinanet-0.5.1
You should consider upgrading via the '/usr/bin/python3 -m pip insta

In [4]:
import tensorflow as tf
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

In [5]:
# root = "/kaggle/input/global-wheat-detection/"
root = 'dataset/'
train_img = root+"train"
test_img = root+"test"
train_csv = "train_annotations.csv"
test_csv = "test_annotations.csv"
sample_submission = root+"sample_submission.csv"

In [6]:
def show_images(images, num = 5):
    
    images_to_show = np.random.choice(images, num)

    for image_id in images_to_show:

        image_path = os.path.join(train_img, image_id + ".jpg")
        image = Image.open(image_path)

        # get all bboxes for given image in [xmin, ymin, width, height]
        bboxes = [literal_eval(box) for box in train[train['image_id'] == image_id]['bbox']]

        # visualize them
        draw = ImageDraw.Draw(image)
        for bbox in bboxes:    
            draw.rectangle([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]], width=3)

        plt.figure(figsize = (15,15))
        plt.imshow(image)
        plt.show()

In [7]:
sorted(os.listdir('snapshots'), reverse=True)[0]

'resnet50_csv_03.h5'

In [8]:
train_csv = pd.read_csv(train_csv)

In [ ]:
PRETRAINED_MODEL = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])
EPOCHS = 5
BATCH_SIZE = 8
STEPS = len(train_csv)//BATCH_SIZE + 1 #Keeping it small for faster commit
LR = 1e-3

# Training Model

In [ ]:
!python3 keras-retinanet/keras_retinanet/bin/train.py --random-transform --backbone resnet50 --compute-val-loss --config config.ini --lr {LR} --batch-size {BATCH_SIZE} --steps {STEPS} --epochs {EPOCHS} --image-min-side 256 --image-max-side 256 csv train_annotations.csv classes.csv --val-annotations test_annotations.csv

Using TensorFlow backend.


2020-05-22 19:53:17.802804: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-05-22 19:53:17.825176: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2208000000 Hz
2020-05-22 19:53:17.826079: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4ac17a0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-05-22 19:53:17.826097: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-05-22 19:53:17.827470: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-22 19:53:17.888137: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-





Epoch 1/5
2020-05-22 19:53:36.472126: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-05-22 19:53:37.154138: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-05-22 19:53:37.814828: W tensorflow/core/common_runtime/bfc_allocator.cc:239] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.28GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2020-05-22 19:53:37.831430: W tensorflow/core/common_runtime/bfc_allocator.cc:239] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2020-05-22 19:53:37.864024: W tensorflow/core/common_runtime/bfc_allocator.cc:239] Allocat

# Loading the trained model

In [ ]:
!ls snapshots

In [ ]:
model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])
model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

# Predictions

In [ ]:
li=os.listdir(test_img)
li[:5]

In [ ]:
def predict(image):
    image = preprocess_image(image.copy())
    image, scale = resize_image(image, min_side=256, max_side=256)
    boxes, scores, labels = model.predict_on_batch(
    np.expand_dims(image, axis=0))
    boxes /= scale
    return boxes, scores, labels

In [ ]:
THRES_SCORE = 0.5

def draw_detections(image, boxes, scores, labels):
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < THRES_SCORE:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(image, b, color=color)

        caption = "{:.3f}".format(score)
        draw_caption(image, b, caption)

In [ ]:
def show_detected_objects(image_name):
    img_path = test_img+'/'+image_name
  
    image = read_image_bgr(img_path)

    boxes, scores, labels = predict(image)
    print(boxes[0,0].shape)
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    draw_detections(draw, boxes, scores, labels)
    plt.figure(figsize=(15,10))
    plt.axis('off')
    plt.imshow(draw)
    plt.show()

In [ ]:
for img in li:
    show_detected_objects(img)

In [ ]:
preds=[]
imgid=[]
for img in tqdm(li,total=len(li)):
    img_path = test_img+'/'+img
    image = read_image_bgr(img_path)
    boxes, scores, labels = predict(image)
    boxes=boxes[0]
    scores=scores[0]
    for idx in range(boxes.shape[0]):
        if scores[idx]>THRES_SCORE:
            box,score=boxes[idx],scores[idx]
            imgid.append(img.split(".")[0])
            preds.append("{} {} {} {} {}".format(score, int(box[0]), int(box[1]), int(box[2]-box[0]), int(box[3]-box[1])))
    

In [ ]:
preds[0]

In [ ]:
sub={"image_id":imgid, "PredictionString":preds}
sub=pd.DataFrame(sub)
sub.head()

In [ ]:
sub_=sub.groupby(["image_id"])['PredictionString'].apply(lambda x: ' '.join(x)).reset_index()
sub_

In [ ]:
# samsub=pd.read_csv("/kaggle/input/global-wheat-detection/sample_submission.csv")
samsub=pd.read_csv("sample_submission.csv")
samsub.head()

In [ ]:
for idx,imgid in enumerate(samsub['image_id']):
    samsub.iloc[idx,1]=sub_[sub_['image_id']==imgid].values[0,1]
    
samsub.head()

In [ ]:
samsub.to_csv('submission.csv',index=False)